In [ ]:
import os
import ast
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from matplotlib.patches import Rectangle

In [ ]:
database = "/home/wmnlab/G/quic_data/"
dates = [
    "2024-07-04",
    "2024-07-17"
]
devices = sorted([
    "sm00",
    "sm01",
])
exps = {
    "QUIC-inf": (3, ["#{:02d}".format(i + 1) for i in range(3)]),
}

exp_duration = 200
data_len = 1223 
time_data = exp_duration * 125000

device_to_port = {"sm00": [5200, 5201], 
                  "sm01": [5202, 5203],
                  "sm02": [5204, 5205],
                  "sm03": [5206, 5207],
                  "sm04": [5208, 5209],
                  "sm05": [5210, 5211],
                  "sm06": [5212, 5213],
                  "sm07": [5214, 5215],
                  "sm08": [5216, 5217],
                  "sm09": [5218, 5219],
                  "MacBookProM1": [4200, 4201],
                  }

Get files.

In [ ]:
def find_ul_sent_file(database, date, exp, device):
    ul_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            file_pair = ["", ""]
            for file in files:
                if file.startswith("ul_processed_sent"):
                    file_pair[0] = os.path.join(root, file)
                if file.startswith("ul_real_lost_pk"):
                    file_pair[1] = os.path.join(root, file)
            ul_files.append(file_pair)
    return ul_files

def find_dl_sent_file(database, date, exp, device):
    dl_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            file_pair = ["", ""]
            for file in files:
                if file.startswith("dl_processed_sent"):
                    file_pair[0] = os.path.join(root, file)
                if file.startswith("dl_real_lost_pk"):
                    file_pair[1] = os.path.join(root, file)
            dl_files.append(file_pair)
    return dl_files

def find_ho_file(database, date, exp, device):
    ho_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            file_pair = ["", "", ""] # LTE, NR, handover_info_log
            for file in files:
                if file.startswith("handover_info_log"):
                    file_pair[0] = os.path.join(root, file)
                if file.startswith("diag_log") & file.endswith("_ml1.csv"):
                    if file.endswith("nr_ml1.csv"):
                        file_pair[2] = os.path.join(root, file)
                    else:
                        file_pair[1] = os.path.join(root, file)
            ho_files.append(file_pair)
    return ho_files

In [ ]:
def HoWithCwnd(ho_df, df):
    df_events = pd.DataFrame(ho_df, columns=['start', 'end'])

    # Ensure timestamps in df_events are in datetime format
    df_events['start'] = pd.to_datetime(df_events['start'])
    df_events['end'] = pd.to_datetime(df_events['end'].fillna(df_events['start']))

    # Prepare the plot
    fig, ax1 = plt.subplots(figsize=(50, 10))

    # Plot congestion window with corrected Timestamp on the primary y-axis
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    ax1.plot(df['Timestamp'], df['congestion_window'], marker='o', markersize=2, color='green', label='Congestion Window')
    ax1.set_xlabel('Timestamp')
    ax1.set_ylabel('Congestion Window', color='green')
    ax1.tick_params(axis='y', labelcolor='green')

    # Create a secondary y-axis to plot the event rectangles
    ax2 = ax1.twinx()
    ax2.set_ylim(ax1.get_ylim())  # Match y-axis limits

    # Draw rectangles for each event section
    for _, event in df_events.iterrows():
        start_time = event['start']
        end_time = event['end']

        # Add a rectangle for 1 second before the event start
        rect_before = Rectangle((start_time - pd.Timedelta(seconds=5), -1), pd.Timedelta(seconds=5), 10, alpha=0.3, color='lightblue', transform=ax2.transData)
        ax2.add_patch(rect_before)

        # Add a rectangle for the event duration (directly using datetime and Timedelta)
        rect = Rectangle((start_time, -1), end_time - start_time, 10, alpha=0.5, color='blue', transform=ax2.transData)
        ax2.add_patch(rect)

        # Add a rectangle for 1 second after the event end
        rect_after = Rectangle((end_time, -1), pd.Timedelta(seconds=5), 10, alpha=0.3, color='lightblue', transform=ax2.transData)
        ax2.add_patch(rect_after)
        
    ax1.set_xlim(df_events['start'].min() - pd.Timedelta(seconds=5), df_events['end'].max() + pd.Timedelta(seconds=5))
    ax2.set_ylim(0, 2.5)  # Set y-limits to ensure rectangles fit well in the plot

    # Set labels and title
    ax1.set_ylabel('Congestion Window', color='green')
    ax1.set_title("Congestion Window & Handover Events")
    ax1.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
def CwndWithPkl(df, lost_df):
    # Create a plot with events marked on the x-axis
    plt.figure(figsize=(50, 10))

    # Set x-axis as timestamp
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Timestamp')

    # Adding vertical lines for lost packets
    for _, row in lost_df.iterrows():
        lost_time = row['Timestamp']
        plt.axvline(x=lost_time, color='brown', linestyle='--', label='Lost Packet')

    # Plot the congestion window
    plt.plot(df['Timestamp'], df['congestion_window'], marker='o', markersize=0.5, color='green', label='Congestion Window')

    plt.title("Congestion Window & Packet Loss")
    plt.tight_layout()
    # plt.savefig(f"{figure_path}/pkl_cwnd_{time}_{port}.png")
    plt.show()

def RttWithLenWithPkl(df, lost_df, time, port):
    # Set the figure size to make the plot wider
    fig, ax1 = plt.subplots(figsize=(50, 10))
    # Plot the 'congestion_window' column on the primary y-axis

    color2 = 'tab:blue'
    ax1.set_ylim(0, 300)
    ax1.set_ylabel('Latest RTT', color=color2)
    ax1.plot(df['timestamp'], df['latest_rtt'], marker='s', markersize=1, color=color2, label='latest_rtt')
    ax1.tick_params(axis='y', labelcolor=color2)

    # Plot the 'length' on the secondary y-axis
    ax2 = ax1.twinx()
    color3 = 'tab:green'
    ax2.plot(df['timestamp'], df['length'], marker='s', markersize=1, color=color3, label='length')
    ax2.tick_params(axis='y', labelcolor=color3)

    # Adding vertical lines for lost packets
    for _, row in lost_df.iterrows():
        lost_time = row['timestamp']
        plt.axvline(x=lost_time, color='brown', linestyle='--', label='Lost Packet')

    # Set title
    plt.title("RTT & Stream Length & Packet Loss")
    plt.tight_layout()
    # plt.savefig(f"{figure_path}/rtt_length_{time}_{port}.png")
    plt.show()

def HoWithPkl(ordered_HOs, lost_df, time, port, comment):
    # Convert the list to a DataFrame
    # df_events = pd.DataFrame(ordered_HOs, columns=['Event', 'Start Time', 'End Time'])
    df_events = pd.DataFrame(ordered_HOs, columns=['Start Time', 'End Time'])

    # Extract timestamp information from HO and MR objects
    df_events['Start Time'] = df_events['Start Time'].apply(lambda x: x.start if hasattr(x, 'start') else (x.time if hasattr(x, 'time') else None))
    df_events['End Time'] = df_events['End Time'].apply(lambda x: x.end if hasattr(x, 'end') else (x.time if hasattr(x, 'time') else None))

    df_events['Start Time'] = pd.to_datetime(df_events['Start Time'])
    df_events['End Time'] = pd.to_datetime(df_events['End Time'].fillna(df_events['Start Time']))

    # Create a plot with events marked on the x-axis
    plt.figure(figsize=(50, 10))
    plt.plot(df_events['Start Time'], [0] * len(df_events), 'o', label='Start Time', markersize=2)
    plt.plot(df_events['End Time'], [1] * len(df_events), 'o', label='End Time', markersize=2)

    # Set x-axis as timestamp
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Timestamp')

    # Add event labels
    # for i, event in df_events.iterrows():
    #     plt.text(event['Start Time'], 0, f'{event["Event"]} (Start)', fontsize=8, ha='right')
    #     plt.text(event['End Time'], 1, f'{event["Event"]} (End)', fontsize=8, ha='right')

    # Draw rectangles for each section
    for _, event in df_events.iterrows():
        start_time = event['Start Time']
        end_time = event['End Time']

        # Add a rectangle for 1 second before the event start
        rect_before = Rectangle((start_time - pd.Timedelta(seconds=1), -0.5), pd.Timedelta(seconds=1), 5, alpha=0.5, color='lightblue')
        plt.gca().add_patch(rect_before)

        # Add a rectangle for the event duration
        rect = Rectangle((start_time, -0.5), end_time - start_time, 5, alpha=1, color='blue')
        plt.gca().add_patch(rect)

        # Add a rectangle for 1 second after the event end
        rect_after = Rectangle((end_time, -0.5), pd.Timedelta(seconds=1), 5, alpha=0.5, color='lightblue')
        plt.gca().add_patch(rect_after)

    # Adding vertical lines for lost packets
    for _, row in lost_df.iterrows():
        if row['trigger'] == "time_threshold":
            lost_time = row['timestamp']
            plt.axvline(x=lost_time, color='brown', linestyle='--', label='Lost Packet')
        else:
            lost_time = row['timestamp']
            plt.axvline(x=lost_time, color='coral', linestyle='--', label='Lost Packet')
        

    plt.title(f"Handover & Packet Loss")
    plt.tight_layout()
    # plt.savefig(f"{figure_path}/{comment}ho_timeline_{time}_{port}.png")
    plt.show()


In [ ]:
for exp in exps:
    for date in dates:
        for device in devices:
            ho_files = find_ho_file(database, date, exp, device)
            ul_sent_files = find_ul_sent_file(database, date, exp, device)
            dl_sent_files = find_dl_sent_file(database, date, exp, device)
            for i, ul_file_pair in enumerate(ul_sent_files):
                print(ul_file_pair)
                ho_df = pd.read_csv(ho_files[i], sep=',')
                ul_sent_df = pd.read_csv(ul_file_pair[0], sep='@')
                ul_lost_df = pd.read_csv(ul_file_pair[1], sep=',')
                HoWithCwnd(ho_df, ul_sent_df)
            for i, dl_file_pair in enumerate(dl_sent_files):
                print(dl_file_pair)
                ho_df = pd.read_csv(ho_files[i], sep=',')
                dl_sent_df = pd.read_csv(dl_file_pair[0], sep='@')
                dl_lost_df = pd.read_csv(dl_file_pair[1], sep=',')
                HoWithCwnd(ho_df, dl_sent_df)

By event

In [ ]:
def filter_rows_around_event(sent_df, event):
    # Ensure the Timestamp column is in datetime format
    sent_df.loc[:, 'Timestamp'] = pd.to_datetime(sent_df.loc[:, 'Timestamp'])

    # Extract event start and end times
    start_time = pd.to_datetime(event['start'])
    end_time = pd.to_datetime(event['end'])

    # Compute the range for filtering
    start_range = start_time - pd.Timedelta(seconds=5)
    end_range = end_time + pd.Timedelta(seconds=5)

    # Filter the DataFrame based on the computed range
    filtered_df = sent_df[(sent_df.loc[:, 'Timestamp'] >= start_range) & (sent_df.loc[:, 'Timestamp'] <= end_range)]

    filtered_df.reset_index()
    return filtered_df

In [ ]:
def HoWithCwndRsrp(ho_df, df):
    df_events = pd.DataFrame(ho_df, columns=['start', 'end', 'lte', 'nr'])

    # Ensure timestamps in df_events are in datetime format
    df_events['start'] = pd.to_datetime(df_events['start'])
    df_events['end'] = pd.to_datetime(df_events.loc[:, 'end'].fillna(df_events['start']))
    df_events['lte'].iloc[0]['Timestamp'] = pd.to_datetime(df_events['lte'].iloc[0]['Timestamp'])
    df_events['nr'].iloc[0]['Timestamp'] = pd.to_datetime(df_events['nr'].iloc[0]['Timestamp'])

    # Prepare the plot
    fig, ax1 = plt.subplots(figsize=(30, 10))

    # Plot congestion window with corrected Timestamp on the primary y-axis
    df.loc[:, 'Timestamp'] = pd.to_datetime(df.loc[:, 'Timestamp'])
    ax1.plot(df['Timestamp'], df['congestion_window'], marker='o', markersize=2, color='green', label='Congestion Window')
    ax1.set_xlabel('Timestamp')
    ax1.set_ylabel('Congestion Window', color='green')
    ax1.tick_params(axis='y', labelcolor='green')

    # Create a secondary y-axis to plot the event rectangles
    ax2 = ax1.twinx()

    # Create a third y-axis
    ax3 = ax1.twinx()
    ax3.plot(df_events['nr'].iloc[0]['Timestamp'], df_events['nr'].iloc[0]['RSRP0'], color='pink', label='RSRP', linewidth=2)
    ax3.plot(df_events['lte'].iloc[0]['Timestamp'], df_events['lte'].iloc[0]['RSRP(dBm)'], color='orange', label='RSRP', linewidth=2)
    ax3.set_ylabel('RSRP', color='orange')
    ax3.tick_params(axis='y', labelcolor='orange')

    # Draw rectangles for each event section
    for _, event in df_events.iterrows():
        start_time = event['start']
        end_time = event['end']

        # Add a rectangle for 1 second before the event start
        rect_before = Rectangle((start_time - pd.Timedelta(seconds=5), -1), pd.Timedelta(seconds=5), 10, alpha=0.3, color='lightblue', transform=ax2.transData)
        ax2.add_patch(rect_before)

        # Add a rectangle for the event duration (directly using datetime and Timedelta)
        rect = Rectangle((start_time, -1), end_time - start_time, 10, alpha=0.5, color='blue', transform=ax2.transData)
        ax2.add_patch(rect)

        # Add a rectangle for 1 second after the event end
        rect_after = Rectangle((end_time, -1), pd.Timedelta(seconds=5), 10, alpha=0.3, color='lightblue', transform=ax2.transData)
        ax2.add_patch(rect_after)
        
    ax1.set_xlim(df_events['start'].min() - pd.Timedelta(seconds=5), df_events['end'].max() + pd.Timedelta(seconds=5))
    ax2.set_ylim(0, 2.5)  # Set y-limits to ensure rectangles fit well in the plot
    ax2.set_yticks([])  # Removes the ticks on the y-axis
    ax2.set_yticklabels([])  # Removes the labels on the y-axis
    # Set labels and title
    ax1.set_ylabel('Congestion Window', color='green')
    ax1.set_title("Congestion Window & Handover Events")
    ax1.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

def HoWithCwndRsrq(ho_df, df):
    df_events = pd.DataFrame(ho_df, columns=['start', 'end', 'lte', 'nr'])

    # Ensure timestamps in df_events are in datetime format
    df_events['start'] = pd.to_datetime(df_events['start'])
    df_events['end'] = pd.to_datetime(df_events.loc[:, 'end'].fillna(df_events['start']))
    df_events['lte'].iloc[0]['Timestamp'] = pd.to_datetime(df_events['lte'].iloc[0]['Timestamp'])
    df_events['nr'].iloc[0]['Timestamp'] = pd.to_datetime(df_events['nr'].iloc[0]['Timestamp'])

    # Prepare the plot
    fig, ax1 = plt.subplots(figsize=(30, 10))

    # Plot congestion window with corrected Timestamp on the primary y-axis
    df.loc[:, 'Timestamp'] = pd.to_datetime(df.loc[:, 'Timestamp'])
    ax1.plot(df['Timestamp'], df['congestion_window'], marker='o', markersize=2, color='green', label='Congestion Window')
    ax1.set_xlabel('Timestamp')
    ax1.set_ylabel('Congestion Window', color='green')
    ax1.tick_params(axis='y', labelcolor='green')

    # Create a secondary y-axis to plot the event rectangles
    ax2 = ax1.twinx()

    # Create a third y-axis
    ax3 = ax1.twinx()
    ax3.plot(df_events['nr'].iloc[0]['Timestamp'], df_events['nr'].iloc[0]['RSRQ0'], color='pink', label='RSRQ (NR)', linewidth=2)
    ax3.plot(df_events['lte'].iloc[0]['Timestamp'], df_events['lte'].iloc[0]['RSRQ(dB)'], color='orange', label='RSRQ', linewidth=2)
    ax3.set_ylabel('RSRQ', color='orange')
    ax3.tick_params(axis='y', labelcolor='orange')

    # Draw rectangles for each event section
    for _, event in df_events.iterrows():
        start_time = event['start']
        end_time = event['end']

        # Add a rectangle for 1 second before the event start
        rect_before = Rectangle((start_time - pd.Timedelta(seconds=5), -1), pd.Timedelta(seconds=5), 10, alpha=0.3, color='lightblue', transform=ax2.transData)
        ax2.add_patch(rect_before)

        # Add a rectangle for the event duration (directly using datetime and Timedelta)
        rect = Rectangle((start_time, -1), end_time - start_time, 10, alpha=0.5, color='blue', transform=ax2.transData)
        ax2.add_patch(rect)

        # Add a rectangle for 1 second after the event end
        rect_after = Rectangle((end_time, -1), pd.Timedelta(seconds=5), 10, alpha=0.3, color='lightblue', transform=ax2.transData)
        ax2.add_patch(rect_after)
        
    ax1.set_xlim(df_events['start'].min() - pd.Timedelta(seconds=5), df_events['end'].max() + pd.Timedelta(seconds=5))
    ax2.set_ylim(0, 2.5)  # Set y-limits to ensure rectangles fit well in the plot
    ax2.set_yticks([])  # Removes the ticks on the y-axis
    ax2.set_yticklabels([])  # Removes the labels on the y-axis
    # Set labels and title
    ax1.set_ylabel('Congestion Window', color='green')
    ax1.set_title("Congestion Window & Handover Events")
    ax1.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
ho_dict = {'MNBH': [], 'SCGM': [], 'MCGF': [], 'SCGF': []}
for exp in exps:
    for date in dates:
        for device in devices:
            ho_files = find_ho_file(database, date, exp, device)
            for file_pair in ho_files:
                ho_df = pd.read_csv(file_pair[0], sep=',')
                lte_df = pd.read_csv(file_pair[1], sep=',')
                nr_df = pd.read_csv(file_pair[2], sep=',')
                mnbh_df = ho_df[ho_df['type'] == 'MNBH']
                for _, event in mnbh_df.iterrows():
                    event['device'] = device
                    lte_filtered_df = filter_rows_around_event(lte_df, event)
                    event['lte'] = [lte_filtered_df]
                    nr_filtered_df = filter_rows_around_event(nr_df, event)
                    event['nr'] = [nr_filtered_df]
                    ho_dict['MNBH'].append(event.to_dict())
                scgm_df = ho_df[ho_df['type'] == 'SCGM']
                for _, event in scgm_df.iterrows():
                    event['device'] = device
                    lte_filtered_df = filter_rows_around_event(lte_df, event)
                    event['lte'] = [lte_filtered_df]
                    nr_filtered_df = filter_rows_around_event(nr_df, event)
                    event['nr'] = [nr_filtered_df]
                    ho_dict['SCGM'].append(event.to_dict())
                mcgf_df = ho_df[ho_df['type'] == 'MCGF']
                for _, event in mcgf_df.iterrows():
                    event['device'] = device
                    lte_filtered_df = filter_rows_around_event(lte_df, event)
                    event['lte'] = [lte_filtered_df]
                    nr_filtered_df = filter_rows_around_event(nr_df, event)
                    event['nr'] = [nr_filtered_df]
                    ho_dict['MCGF'].append(event.to_dict())
                scgf_df = ho_df[ho_df['type'] == 'SCGF']
                for _, event in scgf_df.iterrows():
                    event['device'] = device
                    lte_filtered_df = filter_rows_around_event(lte_df, event)
                    event['lte'] = [lte_filtered_df]
                    nr_filtered_df = filter_rows_around_event(nr_df, event)
                    event['nr'] = [nr_filtered_df]
                    ho_dict['SCGF'].append(event.to_dict())

print("MNBH:", len(ho_dict['MNBH']), "SCGM:", len(ho_dict['SCGM']), "MCGF:", len(ho_dict['MCGF']), "SCGF:", len(ho_dict['SCGF']))

In [ ]:
all_dl_sent_df = None
dl_sent_dfs = []
for exp in exps:
    for date in dates:
        for device in devices:
            # ul_sent_files = find_ul_sent_file(database, date, exp, device)
            dl_sent_files = find_dl_sent_file(database, date, exp, device)
            for file_pair in dl_sent_files:
                dl_sent_df = pd.read_csv(file_pair[0], sep='@')
                dl_sent_df['device'] = device
                dl_sent_dfs.append(dl_sent_df)

all_dl_sent_df = pd.concat(dl_sent_dfs, ignore_index=True)
all_dl_sent_df.reset_index(drop=True, inplace=True)

In [ ]:
for i in range(50, 60):
    dl_sent_df = all_dl_sent_df[all_dl_sent_df['device'] == ho_dict['MNBH'][i]['device']]
    filtered_df = filter_rows_around_event(dl_sent_df, ho_dict['MNBH'][i])
    event_df = pd.DataFrame(ho_dict['MNBH'][i], index=[0])
    # HoWithCwnd(event_df, filtered_df)
    HoWithCwndRsrq(event_df, filtered_df)

In [ ]:
for i in range(50, 55):
    dl_sent_df = all_dl_sent_df[all_dl_sent_df['device'] == ho_dict['SCGM'][i]['device']]
    filtered_df = filter_rows_around_event(dl_sent_df, ho_dict['SCGM'][i])
    event_df = pd.DataFrame(ho_dict['SCGM'][i], index=[0])
    # HoWithCwnd(event_df, filtered_df)
    HoWithCwndRsrq(event_df, filtered_df)

In [ ]:
for i in range(10):
    dl_sent_df = all_dl_sent_df[all_dl_sent_df['device'] == ho_dict['MCGF'][i]['device']]
    filtered_df = filter_rows_around_event(dl_sent_df, ho_dict['MCGF'][i])
    event_df = pd.DataFrame(ho_dict['MCGF'][i], index=[0])
    # HoWithCwnd(event_df, filtered_df)
    HoWithCwndRsrq(event_df, filtered_df)

In [ ]:
mcgf_lost_list = []
for i in range(len(ho_dict['MCGF'])):
    dl_sent_df = all_dl_sent_df[all_dl_sent_df['device'] == ho_dict['MCGF'][i]['device']]
    filtered_df = filter_rows_around_event(dl_sent_df, ho_dict['MCGF'][i])
    # for _, row in filtered_df.iterrows():
    #     if pd.isna(row['congestion_window']):
    #         print("yesy")
    #         break
    # event_df = pd.DataFrame(ho_dict['MCGF'][i], index=[0])
    # Identify the NaN values and calculate the length of consecutive NaNs
    filtered_df.loc[:, 'is_nan'] = filtered_df['congestion_window'].isna()
    filtered_df.loc[:, 'nan_group'] = filtered_df['is_nan'].cumsum()
    filtered_df.loc[:, 'nan_streak'] = filtered_df.groupby((filtered_df['is_nan'] != filtered_df['is_nan'].shift()).cumsum())['is_nan'].transform('sum')
    # Find the largest NaN streak
    largest_nan_streak = filtered_df[filtered_df['is_nan']]['nan_streak'].max()
    # Extract timestamps associated with the largest NaN streak
    timestamps_largest_nan_streak = filtered_df[(filtered_df['is_nan']) & (filtered_df['nan_streak'] == largest_nan_streak)]['Timestamp']

    # Calculate the time difference
    if len(timestamps_largest_nan_streak) > 1:
        time_diff = timestamps_largest_nan_streak.max() - timestamps_largest_nan_streak.min()
    else:
        time_diff = pd.Timedelta(0)

    # Get the index of the first and last NaN in the largest streak
    largest_nan_streak_idx = filtered_df[(filtered_df['is_nan']) & (filtered_df['nan_streak'] == largest_nan_streak)].index
    # Get congestion_window values before and after the largest NaN streak
    before_nan_value = after_nan_value = None
    if not largest_nan_streak_idx.empty and largest_nan_streak_idx.min() > filtered_df.index[0]:
        try:
            before_nan_value = filtered_df.loc[largest_nan_streak_idx.min() - 1, 'congestion_window']
        except:
            before_nan_value = None
    if not largest_nan_streak_idx.empty and largest_nan_streak_idx.max() < filtered_df.index[-1]:
        try:
            after_nan_value = filtered_df.loc[largest_nan_streak_idx.max() + 1, 'congestion_window']
        except:
            after_nan_value = None

    if not before_nan_value == None and not after_nan_value == None:
        cwnd_diff_ratio = (before_nan_value - after_nan_value) * 100 / before_nan_value
    else:
        cwnd_diff_ratio = None
    
    # print(before_nan_value, after_nan_value, cwnd_diff_ratio)
    mcgf_lost_list.append([largest_nan_streak, time_diff, before_nan_value, after_nan_value, cwnd_diff_ratio])

In [ ]:
mcgf_lost_df = pd.DataFrame(mcgf_lost_list, columns=['lost_pkt_cnt', 'lost_pkt_interval', 'cwnd_before', 'cwnd_after', 'cwnd_diff_ratio'])
lost_pkt_cnt_stats = mcgf_lost_df['lost_pkt_cnt'].describe()
lost_pkt_interval_stats = mcgf_lost_df['lost_pkt_interval'].describe()
cwnd_diff_ratio_stats = mcgf_lost_df['cwnd_diff_ratio'].describe()
print(lost_pkt_cnt_stats)
print(lost_pkt_interval_stats)
print(cwnd_diff_ratio_stats)

In [ ]:
import pandas as pd

# Example DataFrame with timestamps
data = {
    'timestamp': pd.date_range(start='2023-01-01', periods=10, freq='T'),
    'congestion_window': [1.2, None, None, 2.3, None, None, 3.4, None, None, 4.5]
}
df = pd.DataFrame(data)

# Identify the NaN values and calculate the length of consecutive NaNs
df['is_nan'] = df['congestion_window'].isna()
df['nan_group'] = df['is_nan'].cumsum()
df['nan_streak'] = df.groupby((df['is_nan'] != df['is_nan'].shift()).cumsum())['is_nan'].transform('sum')
print(df)
# Find the largest NaN streak
largest_nan_streak = df[df['is_nan']]['nan_streak'].max()

# Extract timestamps associated with the largest NaN streak
timestamps_largest_nan_streak = df[(df['is_nan']) & (df['nan_streak'] == largest_nan_streak)]['timestamp']

# Calculate the time difference
if len(timestamps_largest_nan_streak) > 1:
    time_diff = timestamps_largest_nan_streak.max() - timestamps_largest_nan_streak.min()
else:
    time_diff = pd.Timedelta(0)

print("The largest number of consecutive NaN values is:", largest_nan_streak)
print("Timestamps associated with the largest NaN streak:")
print(timestamps_largest_nan_streak)
print("Time difference between the first and last timestamp:", time_diff)


In [ ]:
for i in range(5):
    dl_sent_df = all_dl_sent_df[all_dl_sent_df['device'] == ho_dict['SCGF'][i]['device']]
    filtered_df = filter_rows_around_event(dl_sent_df, ho_dict['SCGF'][i])
    event_df = pd.DataFrame(ho_dict['SCGF'][i], index=[0])
    # HoWithCwnd(event_df, filtered_df)
    HoWithCwndRsrq(event_df, filtered_df)